In [1]:
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import GPT2ForSequenceClassification

In [ ]:
gpt_seq_classifier = GPT2ForSequenceClassification.from_pretrained(
    "openai-community/gpt2"
).to(device="cuda:0")
gpt_seq_classifier

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [25]:
rand_tensor = torch.randn((1, 10, 768)).to(device="cuda:0")
rand_tensor.shape

torch.Size([1, 10, 768])

In [32]:
with torch.no_grad():
    out_logits = gpt_seq_classifier(inputs_embeds=rand_tensor)
out_logits.logits, out_logits.logits.shape

(tensor([[ 6.1311, -5.3402]], device='cuda:0'), torch.Size([1, 2]))

In [1]:
from datasets import load_dataset

train_dataset = load_dataset("tyqiangz/multilingual-sentiments", "all", split="test")
train_dataset[10]
next(iter(train_dataset))

{'text': 'Sepatutnya berbuat begitu\xa0 demi untuk menawarkan sesuatu yang lebih baik kepada rakyat.',
 'source': 'malaya',
 'language': 'malay',
 'label': 0}

In [2]:
from sentiment_dataset import SentimentDataset
from model import load_tokenizer, load_model
from utils import read_yaml, get_tokenizer_config

/home/harsh/anaconda3/envs/DL/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
config = read_yaml("./configs/train_config.yaml")

tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(
    tokenizer_name=tokenizer_name,
    tokenizer_path=tokenizer_path,
    tokenizer_config=tokenizer_config,
)

In [11]:
sentiment_dataset = SentimentDataset(
    dataset=train_dataset,
    tokenizer=tokenizer,
    return_tensors="pt",
)
next(iter(sentiment_dataset))

(tensor([[19117,   265,   315,  3281,    64, 18157, 11110,   265,  4123, 34272,
           1849,  1357,    72,  1418,  2724,  1450,   707,   668,   272,   264,
            274,    84, 33419,   331,   648,   443,    65,  4449, 26605,  1134,
            885,    79,  4763,   374, 15492,   265,    13]]),
 tensor(0))

In [14]:
from datasets import load_dataset

english_dataset = load_dataset("tyqiangz/multilingual-sentiments", 'english', split='train')
english_dataset[-1]


{'text': '"Cant wait to watch the debate tomorrow night. If Romney grills Obama as hard as he did Gingrich in FL, Romney will wipe the floor with Obama" ',
 'source': 'sem_eval_2017',
 'label': 0}

In [18]:
all_dataset = load_dataset("tyqiangz/multilingual-sentiments", 'all', split='train')
all_dataset[0]

{'text': 'yang memerlukan pemerhatian dan tindakan serius',
 'source': 'malaya',
 'language': 'malay',
 'label': 0}

In [24]:
import pandas as pd

In [25]:
df = pd.DataFrame(all_dataset.to_dict())

In [27]:
df['language'].value_counts()

language
japanese      120000
chinese       120000
indonesian     11000
malay           4687
hindi           1839
german          1839
italian         1839
english         1839
portuguese      1839
french          1839
spanish         1839
arabic          1839
Name: count, dtype: int64

In [22]:
for i in all_dataset:
    if i['language']=='malaya':
        out = i
        break

In [21]:
out

{'text': 'okay i\\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\\u2019mon America aren\\u2019t you sick of her yet? (sorry) ',
 'source': 'sem_eval_2017',
 'language': 'english',
 'label': 2}

In [3]:
trainer.evaluate(eval_dataset=val_dataset)

NameError: name 'trainer' is not defined

In [4]:
from sentiment_dataset import load_sentiment_dataset
dataset_temp = load_sentiment_dataset(dataset_path, dataset_language, split)
index = 6
dataset_temp[index]

NameError: name 'dataset_path' is not defined

In [11]:
from model import predict

prediction_config, prompt_samples = get_predicbtion_config(config, pop_samples=True)
decoded_outputs = predict(
    model, tokenizer, dataset_temp[index]["text"], **prediction_config
)

# Llama

In [3]:
!pip install -q accelerate peft bitsandbytes trl

In [1]:
import torch
from transformers import LlamaForSequenceClassification, LlamaTokenizer, BitsAndBytesConfig

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
################################################################################
# bitsandbytes parameters
################################################################################ 
use_4bit = True # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16" # Compute dtype for 4-bit base models 
bnb_4bit_quant_type = "nf4" # Quantization type (fp4 or nf4)
use_nested_quant = False # Activate nested quantization for 4-bit base models (double quantization)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [4]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [ ]:
model = LlamaForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    num_labels=3,
    device_map="cuda:0",
)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig, prepare_model_for_kbit_training

In [7]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="SEQ_CLS",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 39,989,248 || all params: 6,647,345,152 || trainable%: 0.6016


In [9]:
prompt = "Hey, how are you?"
inputs = tokenizer(prompt, return_tensors="pt")

In [12]:
with torch.no_grad():
    outputs = model(inputs.input_ids.to(device='cuda:0'))
outputs.logits.shape

torch.Size([1, 3])

In [13]:
# generate_ids = model.generate(inputs.input_ids.to(device='cuda:0'), max_length=100)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [10]:
model.pad_token_id

2

# Trainer 

In [1]:
import os
import random
from argparse import ArgumentParser
import logging

from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate

from utils import (
    read_yaml,
    get_split_config,
    get_dataset_config,
    get_trainer_config,
    get_model_config,
    get_tokenizer_config,
    get_lora_config,
    _handle_seed,
    DEVICE,
)
from sentiment_dataset import SentimentDataset
from model import load_model, load_tokenizer, predict, compute_metrics, load_lora_model

METRICS_DICT = dict(accuracy=evaluate.load('accuracy'), auc=evaluate.load('roc_auc', 'multiclass'))

In [22]:
config = read_yaml("./configs/train_config_llama.yaml")
config['model_config']

{'model_name': 'llama2-base',
 'base_model_path': 'meta-llama/Llama-2-7b-hf',
 'num_labels': 3,
 'force_download': False,
 'device_map': 'cuda:0',
 'bnb_config': {'load_in_4bit': True,
  'bnb_4bit_quant_type': 'nf4',
  'bnb_4bit_compute_dtype': 'float16',
  'bnb_4bit_use_double_quant': False}}

In [9]:
tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(
    tokenizer_name=tokenizer_name,
    tokenizer_path=tokenizer_path,
    tokenizer_config=tokenizer_config,
)

In [3]:
model_name, model_path, base_model_path, model_config = get_model_config(config)
model_name, model_path, base_model_path, model_config

('llama2-base',
 None,
 'meta-llama/Llama-2-7b-hf',
 {'num_labels': 3,
  'force_download': False,
  'device_map': 'cuda:0',
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_quant_type': 'nf4',
   'bnb_4bit_compute_dtype': 'float16',
   'bnb_4bit_use_double_quant': False}})

In [5]:
model = load_model(
    model_string=model_name,
    model_path=model_path,
    base_model_path=base_model_path,
    model_config=model_config,
)
model.config.pad_token_id = tokenizer.pad_token_id
model_config

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'num_labels': 3, 'force_download': False, 'device_map': 'cuda:0'}

In [23]:
dataset_desc, (train_split_config, val_split_config, test_split_config) = get_split_config(config)
train_split_config, val_split_config

({'dataset_path': 'tyqiangz/multilingual-sentiments',
  'dataset_language': 'english',
  'return_dict': True,
  'padding': 'longest',
  'max_length': 512,
  'sub_split_size': 0.7},
 {'dataset_path': 'tyqiangz/multilingual-sentiments',
  'dataset_language': 'english',
  'return_dict': True,
  'padding': 'longest',
  'max_length': 512,
  'sub_split_size': 0.1})

In [24]:
train_dataset_path, train_dataset_language, train_dataset_config = get_dataset_config(
    train_split_config
)
train_dataset = SentimentDataset(
    dataset_path=train_dataset_path,
    dataset_language='all',
    split_type="train",
    tokenizer=tokenizer,
    **train_dataset_config,
)

In [25]:
import pandas as pd
df = pd.DataFrame(train_dataset.dataset.to_dict())

In [28]:
len(df['language'].value_counts())

12

In [8]:
val_dataset_path, val_dataset_language, val_dataset_config = get_dataset_config(
    val_split_config
)
val_dataset = SentimentDataset(
    dataset_path=val_dataset_path,
    dataset_language=val_dataset_language,
    split_type="validation",
    tokenizer=tokenizer,
    **val_dataset_config,
)

In [9]:
lora_config = get_lora_config(config)
model = load_lora_model(model, lora_config)

In [10]:
model.print_trainable_parameters()

trainable params: 39,989,248 || all params: 6,647,345,152 || trainable%: 0.6016


In [11]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=train_dataset_config.get("padding", True),
    return_tensors="pt"
)

In [12]:
trainer_config = get_trainer_config(config)
trainer_config["logging_dir"] = os.path.join(
    trainer_config["output_dir"], "runs", trainer_config["run_name"]
)
save_trained_model = trainer_config.pop("save_trained_model", True)
trainer_args = TrainingArguments(**trainer_config)

In [13]:
import torch

inputs = tokenizer("hello world", return_tensors="pt")
with torch.no_grad():
    outs = model(inputs.input_ids.to(device="cuda:0"))
outs.logits

tensor([[-2.1068, -0.5030,  2.0381]], device='cuda:0')

In [14]:
trainer = Trainer(
    model,
    args=trainer_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=lambda x: (compute_metrics(x, METRICS_DICT)),
)
training_outs = trainer.train()

  0%|          | 0/516 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/harsh/anaconda3/envs/DL/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


{'loss': 2.5327, 'grad_norm': inf, 'learning_rate': 0.0, 'epoch': 0.01}
{'loss': 2.4203, 'grad_norm': inf, 'learning_rate': 0.0, 'epoch': 0.02}
{'loss': 1.6246, 'grad_norm': 24.94619369506836, 'learning_rate': 3.846153846153847e-06, 'epoch': 0.02}
{'loss': 2.3152, 'grad_norm': 86.29071807861328, 'learning_rate': 1.153846153846154e-05, 'epoch': 0.03}
{'loss': 1.6538, 'grad_norm': 79.77373504638672, 'learning_rate': 1.923076923076923e-05, 'epoch': 0.04}
{'loss': 2.6576, 'grad_norm': 50.0856819152832, 'learning_rate': 2.6923076923076923e-05, 'epoch': 0.05}
{'loss': 1.475, 'grad_norm': 26.794532775878906, 'learning_rate': 3.461538461538462e-05, 'epoch': 0.05}
{'loss': 2.2248, 'grad_norm': 72.29523468017578, 'learning_rate': 4.230769230769231e-05, 'epoch': 0.06}
{'loss': 1.735, 'grad_norm': 28.304706573486328, 'learning_rate': 5e-05, 'epoch': 0.07}
{'loss': 1.7358, 'grad_norm': 58.76654815673828, 'learning_rate': 5.769230769230769e-05, 'epoch': 0.08}
{'loss': 2.4394, 'grad_norm': 75.8490524

KeyboardInterrupt: 

In [21]:
model.save_pretrained(os.path.join(trainer_config["logging_dir"], "model"))

# Inference

In [1]:
# import torch
# from transformers import LlamaForSequenceClassification, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
# tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
# tokenizer.pad_token = tokenizer.eos_token
# ################################################################################
# # bitsandbytes parameters
# ################################################################################ 
# use_4bit = True # Activate 4-bit precision base model loading
# bnb_4bit_compute_dtype = "float16" # Compute dtype for 4-bit base models 
# bnb_4bit_quant_type = "nf4" # Quantization type (fp4 or nf4)
# use_nested_quant = False # Activate nested quantization for 4-bit base models (double quantization)

# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )
# raw_model = LlamaForSequenceClassification.from_pretrained(
#     "meta-llama/Llama-2-7b-hf",
#     quantization_config=bnb_config,
#     num_labels=3,
#     pad_token_id=tokenizer.pad_token_id,
#     device_map="cuda:0",
# )
# # Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)
# from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig, prepare_model_for_kbit_training
# model = PeftModel.from_pretrained(
#     raw_model,
#     "/home/harsh/Desktop/Projects/LLMs/SentimentClassification/TrainingLogs/runs/llama_test/model",
# )
# model = model.merge_and_unload()
# # lora_r = 16
# # lora_alpha = 64
# # lora_dropout = 0.1
# # lora_target_modules = [
# #     "q_proj",
# #     "up_proj",
# #     "o_proj",
# #     "k_proj",
# #     "down_proj",
# #     "gate_proj",
# #     "v_proj",
# # ]


# # peft_config = LoraConfig(
# #     r=lora_r,
# #     lora_alpha=lora_alpha,
# #     lora_dropout=lora_dropout,
# #     target_modules=lora_target_modules,
# #     bias="none",
# #     task_type="SEQ_CLS",
# # )

# # model = prepare_model_for_kbit_training(model)
# # model = get_peft_model(model, peft_config)
# # model.print_trainable_parameters()

In [32]:
import torch

prompt = "Just listening to whitney houston, celin dione and mariah carey.... just a perfect sunday,, true talent there ladies and gents"
inputs = tokenizer(prompt, return_tensors="pt")

In [33]:
with torch.no_grad():
    outputs = model(inputs.input_ids.to(device='cuda:0'))
outputs.logits.shape, outputs.logits.argmax(dim=-1), train_dataset.index2label

(torch.Size([1, 3]),
 tensor([0], device='cuda:0'),
 {0: 'positive', 1: 'neutral', 2: 'negative'})

In [34]:
train_dataset.dataset[1]

{'text': 'Just listening to whitney houston, celin dione and mariah carey.... just a perfect sunday,, true talent there ladies and gents ',
 'source': 'sem_eval_2017',
 'label': 0}

In [1]:
import os
import random
from argparse import ArgumentParser
import logging

import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

from utils import (
    read_yaml,
    get_model_config,
    get_tokenizer_config,
    get_split_config,
    get_dataset_config,
    get_trainer_config,
    get_prediction_config,
    get_lora_config,
    _handle_seed,
    DEVICE,
)
from sentiment_dataset import SentimentDataset
from model import (
    load_model,
    load_tokenizer,
    load_lora_model,
    predict,
    compute_metrics,
    METRICS_DICT,
)


In [2]:
config = read_yaml("./configs/predict_config_llama.yaml")
config

{'dataset_config': {'desc': 'Multilinuagal sentiment classification dataset',
  'test': {'dataset_path': 'tyqiangz/multilingual-sentiments',
   'dataset_language': 'english',
   'return_dict': True,
   'padding': 'longest',
   'max_length': 512,
   'sub_split_size': None}},
 'tokenizer_config': {'tokenizer_name': 'llama2',
  'tokenizer_path': 'meta-llama/Llama-2-7b-hf'},
 'model_config': {'model_name': 'llama2-finetuned',
  'model_path': '/home/harsh/Desktop/Projects/LLMs/SentimentClassification/TrainingLogs/runs/llama2_english/model',
  'base_model_path': 'meta-llama/Llama-2-7b-hf',
  'num_labels': 3,
  'force_download': False,
  'device_map': 'cuda:0',
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_quant_type': 'nf4',
   'bnb_4bit_compute_dtype': 'float16',
   'bnb_4bit_use_double_quant': False}},
 'prediction_config': {'samples': ['what was the irony?',
   'Gisburn was very curious']}}

In [3]:
tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(
    tokenizer_name=tokenizer_name,
    tokenizer_path=tokenizer_path,
    tokenizer_config=tokenizer_config,
)

In [4]:
model_name, model_path, base_model_path, model_config = get_model_config(config)
model_config.update(dict(pad_token_id=tokenizer.pad_token_id))

In [5]:
model = load_model(
    model_string=model_name,
    model_path=model_path,
    base_model_path=base_model_path,
    model_config=model_config,
)
model = model.to(device=DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
import os
import random
from argparse import ArgumentParser
import logging

import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

from utils import (
    read_yaml,
    get_model_config,
    get_tokenizer_config,
    get_split_config,
    get_dataset_config,
    get_trainer_config,
    get_prediction_config,
    get_lora_config,
    _handle_seed,
    DEVICE,
)
from sentiment_dataset import SentimentDataset
from model import (
    load_model,
    load_tokenizer,
    load_lora_model,
    predict,
    compute_metrics,
    METRICS_DICT,
)


In [2]:
config = read_yaml("./configs/train_config_gpt2.yaml")

In [3]:
tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(
    tokenizer_name=tokenizer_name,
    tokenizer_path=tokenizer_path,
    tokenizer_config=tokenizer_config,
)

In [4]:
# dataset_desc, (train_split_config, val_split_config, test_split_config) = (
#     get_split_config(config)
# )

# test_dataset_path, test_dataset_language, test_dataset_config = (
#     get_dataset_config(test_split_config)
# )
# test_dataset = SentimentDataset(
#     dataset_path=test_dataset_path,
#     dataset_language=test_dataset_language,
#     split_type="test",
#     tokenizer=tokenizer,
#     **test_dataset_config,
# )

In [5]:
model_name, model_path, base_model_path, model_config = get_model_config(config)
model_config.update(dict(pad_token_id=tokenizer.pad_token_id))
model_name, model_path, base_model_path, model_config

('gpt2',
 'openai-community/gpt2',
 None,
 {'num_labels': 3, 'pad_token_id': 50256})

In [6]:
model = load_model(
    model_string=model_name,
    model_path=model_path,
    base_model_path=base_model_path,
    model_config=model_config,
)
model = model.to(device=DEVICE)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
